In [ ]:
pip install gradio faiss-cpu sentence-transformers requests langchain


: 

In [ ]:
# ========================================
# 🏫 School Parent Helpdesk Chatbot (Gradio + FAISS + NVIDIA RAG)
# Works directly in Google Colab
# ========================================

# 🔹 Install dependencies
!pip install -q gradio faiss-cpu sentence-transformers requests langchain pypdf2 python-docx pandas

# ------------------------------
# Imports
# ------------------------------
import os
import gradio as gr
import faiss
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from PyPDF2 import PdfReader
import docx

# ------------------------------
# NVIDIA API Settings
# ------------------------------
NVIDIA_API_KEY = "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"   # 🔑 replace with your NVIDIA key
NVIDIA_CHAT_URL = "https://integrate.api.nvidia.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "Content-Type": "application/json"
}

# ------------------------------
# Embedding + FAISS Setup
# ------------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

def load_file(file_path):
    """Reads text from PDF, DOCX, TXT, or CSV"""
    ext = file_path.split(".")[-1].lower()
    text = ""
    if ext == "pdf":
        reader = PdfReader(file_path)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif ext == "docx":
        doc = docx.Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
    elif ext == "csv":
        df = pd.read_csv(file_path)
        text = df.astype(str).to_string()
    else:  # assume txt
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
    return text

def build_faiss_index(docs):
    """Split docs → create embeddings → build FAISS index"""
    chunks, texts = [], []
    for doc in docs:
        parts = splitter.split_text(doc)
        chunks.extend([Document(page_content=p) for p in parts])
    texts = [c.page_content for c in chunks]

    vectors = embedder.encode(texts)
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)

    return index, texts

# ------------------------------
# NVIDIA Chat Function
# ------------------------------
def nvidia_llm(query, context=""):
    payload = {
        "model": "meta/llama-3.1-8b-instruct",
        "messages": [
            {"role": "system", "content": "You are a helpful school parent helpdesk assistant."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
        ],
        "temperature": 0.3
    }
    resp = requests.post(NVIDIA_CHAT_URL, headers=headers, json=payload)
    return resp.json()["choices"][0]["message"]["content"]

# ------------------------------
# RAG Pipeline (Helpdesk Agent)
# ------------------------------
index, texts = None, []

def rag_agent(query, history):
    global index, texts
    if index is None:
        answer = "⚠️ Please upload the school helpdesk PDF first."
        history.append((query, answer))
        return history, history

    q_vec = embedder.encode([query])
    D, I = index.search(q_vec, k=3)
    context = "\n".join([texts[i] for i in I[0]])

    # Answer with context
    answer = nvidia_llm(query, context)
    history.append((query, answer))
    return history, history

def upload_and_build(file):
    global index, texts
    text = load_file(file.name)
    index, texts = build_faiss_index([text])
    return f"✅ Knowledge base built from {os.path.basename(file.name)}"

# ------------------------------
# Gradio UI
# ------------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🏫 School Parent Helpdesk Chatbot")

    with gr.Row():
        file_upload = gr.File(label="📂 Upload School Document", file_types=[".pdf", ".txt", ".docx", ".csv"])
        upload_status = gr.Textbox(label="Upload Status")

    file_upload.upload(upload_and_build, file_upload, upload_status)

    chat_ui = gr.Chatbot(label="💬 Ask about school facilities, enrollment, exam dates...")
    msg = gr.Textbox(label="Type your question here...")
    clear = gr.Button("Clear Chat")

    state = gr.State([])

    msg.submit(rag_agent, [msg, state], [chat_ui, state])
    clear.click(lambda: ([], []), None, [chat_ui, state])

demo.launch(inline=True)  # Works in Colab
